<span style='color:red'> This jupyter-notebook contains solutions for the Practice Quizes attempted while completign the Project: Investigate a Relational Database. </span>

***

## <span style='color:blue'> Practice Quiz #1 </span>

**Q1**: Create a table that provides the following details: 
    - Actor's first and last name combined as full_name
    - Film title
    - Film description
    - Length of the movie. 

```sql

SELECT  a.first_name AS actor_firstname,
		a.last_name AS actor_lastname,
		a.first_name || ' ' || a.last_name AS actor_fullname,
        f.title AS film_title,
        f.description AS film_description,
        f.length AS film_length
FROM film_actor AS fa
JOIN actor AS a
ON a.actor_id = fa.actor_id
JOIN film AS f
ON f.film_id = fa.film_id;

```

**Q2**: Create a table with list of actors and movies where the movie length was more than 60 minutes. 
    - How many rows are there in this query result?

```sql
SELECT COUNT(*) nfilms_gt60
FROM
    (SELECT  a.first_name || ' ' || a.last_name AS actor_fullname,
            f.title AS film_title,
            f.length AS film_length
    FROM film_actor AS fa
    JOIN actor AS a
    ON a.actor_id = fa.actor_id
    JOIN film AS f
    ON f.film_id = fa.film_id
    WHERE f.length > 60
    ) sq;
```

**Q3**: Write a query that captures the 
    - actor id
    - full name of the actor
    - counts the number of movies each actor has made
    (HINT: Think about whether you should group by actor id or the full name of the actor.) 
    - Identify the actor who has made the maximum number movies.

```sql

SELECT  a.actor_id AS actor_id,
        a.first_name || ' ' || a.last_name AS actor_fullname,
        COUNT(f.title) AS nfilms_actor
FROM film_actor AS fa
JOIN actor AS a
ON a.actor_id = fa.actor_id
JOIN film AS f
ON f.film_id = fa.film_id
GROUP BY 1
ORDER BY 3 DESC
LIMIT 1;

```

***

## <span style='color:blue'> Practice Quiz #2 </span>

**Q1**: Write a query that displays a table with 4 columns: 
    - actor's full name
    - film title
    - length of movie
    - a column name "filmlen_groups" that classifies movies based on their length. 
    - Filmlen_groups should include 4 categories: 
        - 1 hour or less
        - Between 1-2 hours
        - Between 2-3 hours
        - More than 3 hours.

```sql

SELECT  a.first_name || ' ' || a.last_name AS actor_fullname,
        f.title AS film_title,
        f.length AS film_length,
        CASE
            WHEN (f.length <= 60) THEN '1 hour or less'
            WHEN (f.length > 60 AND f.length <= 120) THEN 'Between 1-2 hours'
            WHEN (f.length > 120 AND f.length <= 180) THEN 'Between 2-3 hours'
            ELSE 'More than 3 hours' 
        END AS filmlen_groups
FROM film_actor AS fa
JOIN actor AS a
ON a.actor_id = fa.actor_id
JOIN film AS f
ON f.film_id = fa.film_id;

```

Q2: Revise the query you wrote above to create a count of movies in each of the 4 filmlen_groups: 1 hour or less, Between 1-2 hours, Between 2-3 hours, More than 3 hours.

```sql

WITH film_groups AS (SELECT f.title,
        f.length,
        CASE
            WHEN (f.length <= 60) THEN '1 hour or less'
            WHEN (f.length > 60 AND f.length <= 120) THEN 'Between 1-2 hours'
            WHEN (f.length > 120 AND f.length <= 180) THEN 'Between 2-3 hours'
            ELSE 'More than 3 hours' 
        END AS filmlen_groups
FROM film AS f)


SELECT DISTINCT(filmlen_groups),
		COUNT(title) OVER (PARTITION BY filmlen_groups) AS nfilm_groups
FROM film_groups

```

***

**Q1**: Create a query that lists each movie, the film category it is classified in, and the number of times it has been rented out.
    - We want to understand more about the movies that families are watching. The following categories are considered family movies: Animation, Children, Classics, Comedy, Family and Music. 

```sql

WITH table_fc AS 
    (SELECT  f.title AS film_title, 
            c.name AS category_name
    FROM film AS f 
    JOIN film_category AS fc
        ON fc.film_id = f.film_id
    JOIN category AS c
        ON c.category_id = fc.category_id),
        
	table_frc AS 
    (SELECT  f.title AS film_title, 
              COUNT(r.inventory_id) OVER (PARTITION BY f.title) AS rental_count
      FROM film AS f 
      JOIN inventory AS i
          ON f.film_id = i.film_id
      JOIN rental AS r
          ON r.inventory_id = i.inventory_id
      )


SELECT DISTINCT(tfrc.film_title), tfc.category_name, tfrc.rental_count
FROM table_frc AS tfrc
JOIN table_fc AS tfc
    ON tfc.film_title = tfrc.film_title
WHERE tfc.category_name LIKE 'Animation'
/*WHERE tfc.category_name LIKE 'Children'
WHERE tfc.category_name LIKE 'Classics'
WHERE tfc.category_name LIKE 'Comedy'
WHERE tfc.category_name LIKE 'Family'
WHERE tfc.category_name LIKE 'Music'
*/
ORDER BY 1;

```

Now we need to know how the length of rental duration of these family-friendly movies compares to the duration that all movies are rented for. 

**Q2**: Provide a table with the movie titles and divide them into 4 levels (first_quarter, second_quarter, third_quarter, and final_quarter) based on the quartiles (25%, 50%, 75%) of the rental duration for movies across all categories? Make sure to also indicate the category that these family-friendly movies fall into.

Check Your Solution: 
The data are not very spread out to create a very fun looking solution, but you should see something like the following if you correctly split your data. You should only need the category, film_category, and film tables to answer this and the next questions.

HINT: One way to solve it requires the use of percentiles, Window functions, subqueries or temporary tables.

```sql

SELECT  f.title AS film_title, 
    	c.name AS category_name,
    	f.rental_duration AS film_rental_duration,
   		NTILE(4) OVER (PARTITION BY f.rental_duration) AS standard_quartile 
FROM film AS f 
JOIN film_category AS fc
	ON fc.film_id = f.film_id
JOIN category AS c
  	ON c.category_id = fc.category_id
WHERE c.name IN ('Animation','Children','Classics','Comedy','Family','Music')
GROUP BY 1,2,3
ORDER BY 3;
  

```

**Q3**: Finally, provide a table with the family-friendly film category, each of the quartiles, and the corresponding count of movies within each combination of film category for each corresponding rental duration category. The resulting table should have three columns:
   - Category
   - Rental length category
   - Count

```sql

WITH table_cq AS 
    (SELECT  c.name category_name,
              NTILE(4) OVER (ORDER BY f.rental_duration) AS quartiles
    FROM film f
    JOIN film_category fc
      ON f.film_id = fc.film_id
    JOIN category c
      ON c.category_id = fc.category_id
    WHERE c.name IN ('Animation', 'Children', 'Classics', 'Comedy', 'Family', 'Music')
    )


SELECT tcq.category_name AS name, 
       tcq.quartiles AS standard_quartile, 
       COUNT(tcq.category_name) AS count
FROM table_cq AS tcq
GROUP BY 1,2
ORDER BY 1,2

```

***

## <span style='color:blue'> Question Set #2 </span>

**Q1:** We want to find out how the two stores compare in their count of rental orders during every month for all the years we have data for. 

Write a query that returns: 
    - the store ID for the store, 
    - the year
    - the month
    - the number of rental orders each store has fulfilled for that month. 
    - Your table should include a column for each of the following: year, month, store ID and count of rental orders fulfilled during that month. 

```sql

SELECT  DATE_PART('month',r.rental_date) AS Rental_month,
		DATE_PART('year',r.rental_date) AS Rental_year,
		s.store_id AS Store_ID,
		COUNT(*) AS Count_rentals
FROM rental AS r
JOIN staff AS s
ON r.staff_id = s.staff_id
GROUP BY 1,2,3
ORDER BY 4 DESC


```

**Q2:** We would like to know who were our top 10 paying customers. How many payments they made on a monthly basis during 2007, and what was the amount of the monthly payments. 

Write a query to capture the customer name, month and year of payment, and total payment amount for each month by these top 10 paying customers?

```sql

WITH customer_payments AS 
        (SELECT DATE_TRUNC('month',p.payment_date) AS pay_month, 
               c.first_name || ' ' || c.last_name AS fullname, 
               COUNT(*) AS pay_countpermon, 
               SUM(p.amount) AS pay_amount
        FROM customer AS c
        JOIN payment AS p
            ON p.customer_id = c.customer_id
        WHERE DATE_TRUNC('month', p.payment_date) > '2006-12-31'
        GROUP BY 1,2
        ORDER BY 2),

     customer_top10 AS
        (SELECT  c.first_name || ' ' || c.last_name AS fullname,
                SUM(p.amount) AS total_amout
        FROM customer AS c 
        JOIN payment AS p
            ON p.customer_id = c.customer_id
        GROUP BY 1
        ORDER BY 2 DESC 
        LIMIT 10)


SELECT  cp.pay_month AS pay_month, 
        cp.fullname AS fullname,
        cp.pay_countpermon AS pay_countpermon,
        cp.pay_amount AS pay_amount
FROM customer_top10 AS c10
JOIN customer_payments AS cp
    ON cp.fullname = c10.fullname
```

**Q3:** Finally, for each of these top 10 paying customers, I would like to find out the difference across their monthly payments during 2007. 

Write a query to compare the payment amounts in each successive month. Repeat this for each of these 10 paying customers. Also, it will be tremendously helpful if you can identify the customer name who paid the most difference in terms of payments.

```sql
WITH customer_payments AS 
        (SELECT DATE_TRUNC('month',p.payment_date) AS pay_month, 
               c.first_name || ' ' || c.last_name AS fullname, 
               COUNT(*) AS pay_countpermon, 
               SUM(p.amount) AS pay_amount
        FROM customer AS c
        JOIN payment AS p
            ON p.customer_id = c.customer_id
        WHERE DATE_TRUNC('month', p.payment_date) > '2006-12-31'
        GROUP BY 1,2
        ORDER BY 2),

     customer_top10 AS
        (SELECT  c.first_name || ' ' || c.last_name AS fullname,
                SUM(p.amount) AS total_amout
        FROM customer AS c 
        JOIN payment AS p
            ON p.customer_id = c.customer_id
        GROUP BY 1
        ORDER BY 2 DESC 
        LIMIT 10),

     customer_month AS 
        (SELECT  cp.pay_month, 
                 cp.fullname, 
                 cp.pay_countpermon, 
                 cp.pay_amount
        FROM customer_payments AS cp
        JOIN customer_top10 AS c10
        ON cp.fullname = c10.fullname
        ORDER BY fullname, pay_month)


SELECT  pay_month,
		fullname,
        pay_amount - LAG(pay_amount) OVER (ORDER BY fullname) AS difference,
        LAG(pay_amount) OVER (ORDER BY fullname) AS lag_column
        FROM customer_month
ORDER BY 3 DESC
LIMIT 2;
    
```